In [2]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import preprocess
from sklearn.model_selection import train_test_split
import scipy.io as sio
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

In [3]:
%%time
# load input files
rootdir = '../../data.nosync/'
interictal_count = preprocess.count_interictal_files(rootdir)
interictal_files = preprocess.get_interictal_files(rootdir, interictal_count)
preictal_files = preprocess.get_preictal_files(rootdir)
print('Interictal file count:', len(interictal_files))
print('Preictal file count:', len(preictal_files))

Interictal file count: 480
Preictal file count: 24
CPU times: user 2.94 ms, sys: 7.6 ms, total: 10.5 ms
Wall time: 12.7 ms


In [4]:
# target vector
interictal_y = np.zeros(len(interictal_files))
preictal_y = np.ones(len(preictal_files))
y = np.concatenate((interictal_y, preictal_y), axis=0)
y.shape

(504,)

In [5]:
%%time
# input vector
X_list = []
X_list.extend(interictal_files)
X_list.extend(preictal_files)
X = np.array(X_list)
print(type(X))
X.shape

<class 'numpy.ndarray'>
CPU times: user 546 µs, sys: 345 µs, total: 891 µs
Wall time: 958 µs


(504,)

In [6]:
# summarize class distribution (0=interictal, 1=pre-ictal)
print(Counter(y))

Counter({0.0: 480, 1.0: 24})


In [7]:
# define oversampling strategy due to class imbalance
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X.reshape(-1,1), y)
print(Counter(y_over))

Counter({0.0: 480, 1.0: 480})


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.33, random_state=42)
print(X_train.shape)
print(X_test.shape)

(643, 1)
(317, 1)


In [9]:
# convert to z-score and flatten training set
X_train_EEG = np.zeros((643, 16, 239766))
for i in tqdm(range(len(X_train))):
    filepath = rootdir + X_train[i].item()
    mat = sio.loadmat(filepath)
    sigbuf = preprocess.get_sig(mat).astype('float32')
    mean, std = sigbuf.mean(), sigbuf.std()
    sigbuf = (sigbuf - mean) / std # normalise
    X_train_EEG[i,:] = sigbuf

100%|██████████| 643/643 [01:44<00:00,  6.15it/s]


In [10]:
# convert to z-score and flatten test set
X_test_EEG = np.zeros((317, 16, 239766))
for i in tqdm(range(len(X_test))):
    filepath = rootdir + X_test[i].item()
    mat = sio.loadmat(filepath)
    sigbuf = preprocess.get_sig(mat).astype('float32')
    mean, std = sigbuf.mean(), sigbuf.std()
    sigbuf = (sigbuf - mean) / std # normalise
    X_test_EEG[i,:] = sigbuf

100%|██████████| 317/317 [00:58<00:00,  5.46it/s]


Architecture is a fully convolutional neural network according to [Wang, 2016](https://arxiv.org/abs/1611.06455).

In [11]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)
    
    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)
    
    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)
    
    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)
    
    return keras.models.Model(inputs=input_layer, outputs=output_layer)

num_classes = 2
model = make_model(input_shape=X_train_EEG.shape[1:])

In [12]:
%%time
epochs = 5
batch_size = 32

callbacks = [
#     keras.callbacks.ModelCheckpoint(
#         "best_model.h5", save_best_only=True, monitor="val_loss"
#     ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(
    X_train_EEG,
    y_train.reshape(-1,1),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/5


ValueError: in user code:

    /Users/sameenislam/anaconda3/envs/ml/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/sameenislam/anaconda3/envs/ml/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/sameenislam/anaconda3/envs/ml/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/sameenislam/anaconda3/envs/ml/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/sameenislam/anaconda3/envs/ml/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs

In [ ]:
# model = keras.models.load_model("best_model.h5")
test_loss, test_acc = model.evaluate(X_test_EEG, y_test)
print("Test accuracy:", test_acc)
print("Test loss:", test_loss)

In [ ]:
metric = "accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")